# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error

In [1]:
const name = "UserItemBiases"
const residual_alphas = []
const implicit = false;

In [2]:
import NBInclude: @nbinclude
import SparseArrays: sparse
import Statistics: mean
@nbinclude("Alpha.ipynb");

In [3]:
const training = get_split("training", implicit)
const validation = get_split("validation", implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge

## ALS for Explicit data
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        @inbounds r[i] = u[users[i]] + a[items[i]]
    end
    r
end;

In [5]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    @inbounds for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)
    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [6]:
function train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training", implicit), log(λ_wu)) .*
        expdecay(get_counts("training", implicit; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    r = make_prediction(validation.user, validation.item, u, a)
    residualized_loss(residual_alphas, implicit, r)
end;

In [8]:
validation_mse(fill(0.0f0, 4))

[ Info: 20220604 15:06:34 training model with parameters Float32[1.0, 1.0, 1.0, 1.0]
Progress: 100%|███████████████████████████| Time: 0:00:02 ( 0.15 μs/it)39m
Progress: 100%|███████████████████████████| Time: 0:00:00 (32.59 ns/it)


1.813418f0

In [9]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.LBFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220604 15:07:04 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]


Iter     Function value   Gradient norm 
     0     1.813422e+00     5.017445e-02
 * Current step size: 1.0
 * time: 0.03074789047241211
 * g(x): Float32[-0.0045934455, -3.0179476f-6, 0.018213015, 0.050174452]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220604 15:07:54 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.004604,1.004604,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.000003,0.0,1.000003,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98195183,0.0,0.0,0.98195183,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9510635,0.0,0.0,0.0,0.9510635)]
[ Info: 20220604 15:08:33 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.023233,1.023233,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000151,0.0,1.0000151,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.91295826,0.0,0.0,0.91295826,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.77812177,0.0,0.0,0.0,0.77812177)]
[ Info: 20220604

     1     1.811669e+00     1.526675e-02
 * Current step size: 1.0271175
 * time: 118.09454584121704
 * g(x): Float32[-0.0010347532, -3.7929474f-6, 0.015266755, -0.0015451695]
 * x: Float32[0.004718008, 3.0997867f-6, -0.018706907, -0.05153506]


[ Info: 20220604 15:09:50 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0062914,1.0062914,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000075,0.0,1.0000075,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9636046,0.0,0.0,0.9636046,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.94699776,0.0,0.0,0.0,0.94699776)]
[ Info: 20220604 15:10:27 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0125649,1.0125649,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000253,0.0,1.0000253,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.89534736,0.0,0.0,0.89534736,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9359882,0.0,0.0,0.0,0.9359882)]
[ Info: 2022

...

    12     1.808033e+00     1.764127e-04
 * Current step size: 2.013646
 * time: 1498.864665031433
 * g(x): Float32[-1.9009552f-5, 1.9087082f-7, 5.1412477f-5, 0.00017641265]
 * x: Float32[-0.34672502, -0.8923888, -0.42899656, -0.01885572]


[ Info: 20220604 15:32:51 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7084238,0.7084238,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.40588373,0.0,0.40588373,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6511278,0.0,0.0,0.6511278,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98142165,0.0,0.0,0.0,0.98142165)]
[ Info: 20220604 15:33:28 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.71414876,0.71414876,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.39106274,0.0,0.39106274,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6509905,0.0,0.0,0.6509905,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9818246,0.0,0.0,0.0,0.9818246)]
[ Info: 

    13     1.808033e+00     5.474516e-06
 * Current step size: 1.1224906
 * time: 1611.0625398159027
 * g(x): Float32[3.724754f-7, 3.8308595f-8, -1.622722f-6, -5.4745155f-6]
 * x: Float32[-0.34446633, -0.9028276, -0.42905575, -0.018740539]


[ Info: 20220604 15:34:43 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.70864666,0.70864666,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.40451136,0.0,0.40451136,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6511299,0.0,0.0,0.6511299,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9814397,0.0,0.0,0.0,0.9814397)]
[ Info: 20220604 15:35:21 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7088397,0.7088397,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.40089065,0.0,0.40089065,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.65115505,0.0,0.0,0.65115505,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98146254,0.0,0.0,0.0,0.98146254)]
[ Info

    14     1.808033e+00     6.412606e-07
 * Current step size: 1.1233808
 * time: 1723.2058458328247
 * g(x): Float32[3.0843395f-9, 1.3546859f-9, -1.0447979f-8, -6.412606f-7]
 * x: Float32[-0.34438983, -0.9053528, -0.4290449, -0.018734004]


In [10]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220604 15:36:35 The optimal λ is Float32[0.7086526, 0.4043992, 0.6511307, 0.98144037], found in 47 function calls


In [11]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ);

[ Info: 20220604 15:36:35 training model with parameters Float32[0.7086526, 0.4043992, 0.6511307, 0.98144037]


In [12]:
validation_mse(Optim.minimizer(res))

[ Info: 20220604 15:36:48 training model with parameters Float32[0.7086526, 0.4043992, 0.6511307, 0.98144037]


1.8080267f0

## Inference

In [13]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, residual_alphas, implicit);

[ Info: 20220604 15:37:03 validation loss: 1.8080267, β: Float32[1.0015497]
[ Info: 20220604 15:37:06 training loss: 1.6717246, β: Float32[1.0015497]


In [14]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));